In [2]:
import io
import os
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras import Input, Model, layers, losses, optimizers, callbacks
import tensorflow_decision_forests as tfdf


default_max_columns = None
default_max_rows = 10
pd.set_option('display.max_columns', default_max_columns)
pd.set_option('display.max_rows', default_max_rows)

df_name = "X-IIoTiD"
test_size_ratio = 0.2
val_size_ratio = 0.1
cwd = os.getcwd()

In [3]:
#Utility functions

def convert_to_number(x,replacement_number):
    try:
        x = int(x)
    except ValueError:
        try:
            x = float(x)
            if np.isnan(x):
                x = replacement_number
        except ValueError:
            x = replacement_number
    return x

def ip_to_octets(ip_adress):
    if ip_adress == "?" or pd.isnull(ip_adress):
        return [replacement_number]*4
    octets = str(ip_adress).split(".")
    if len(octets) != 4:
        raise Exception("Invalid IP Adress: "+str(ip_adress))
    else:
        for i in range(0,3):
            octets[i] = int(octets[i])
        return octets

def ip_to_octet(ip_adress, octet_index):
    octets = ip_to_octets(ip_adress)
    return octets[octet_index]

def frame_ip_to_octet(input_frame, ip_column, cleanup = True):
    frame = input_frame.copy()
    for i in range(4):
        frame[ip_column+"_octet"+str(i)] = frame[ip_column].apply(lambda x : ip_to_octet(x,i)).astype("float32")
    if cleanup:
        frame = frame.drop([ip_column],axis = 1)
    return frame


#Imputes by sampling mean and std derived from frames that have been split by unique values from a target column
#Most data is very dependent on protocol type
def impute_by_target_col(input_frame, target_col, cols_to_impute):
    output_frame = pd.DataFrame()
    for value in input_frame[target_col].unique():
        df_temp = input_frame.loc[input_frame[target_col] == value].copy() 
        for col in df_temp.columns:
            if col == target_col or col not in cols_to_impute:
                continue
            mean, std = df_temp[col].mean(), df_temp[col].std()
            if pd.isna(mean) or pd.isna(std):
                mean, std = input_frame[col].mean(), input_frame[col].std()
            df_temp[col] = df_temp[col].apply(lambda x: np.random.normal(mean,std) if pd.isna(x) else x)
        output_frame = pd.concat([output_frame,df_temp],ignore_index = True)
    return output_frame

#Removes all Features that do no Intersect with other datasets
def remove_features(input_frame): 
    ftk = features_to_keep.copy()
    for column in input_frame.columns:
        for target_col in ["Src_IP","Des_IP","Src_Port","Des_Port","Protocol"]:
            if target_col in column:
                ftk.append(column)
                break;              
    return input_frame[ftk]

In [4]:
#Load Data
file_path = "" #File paths deleted for privacy reasons

df_raw = pd.read_csv(file_path, dtype=str)

In [5]:
#Columns that need cleaning sorted by data types
num_cols = ["Scr_port",
            "Des_port",
            "Duration",
            "Scr_bytes",
            "Des_bytes",
            "missed_bytes",
            "Scr_pkts",
            "Scr_ip_bytes",
            "Des_pkts",
            "Des_ip_bytes",
            "total_bytes",
            "total_packet",
            "paket_rate",
            "byte_rate",
            "Scr_packts_ratio",
            "Des_pkts_ratio",
            "Scr_bytes_ratio",
            "Des_bytes_ratio",
            "Avg_user_time",
            "Std_user_time",
            "Avg_nice_time",
            "Std_nice_time",
            "Avg_system_time",
            "Std_system_time",
            "Avg_iowait_time",
            "Std_iowait_time",
            "Avg_ideal_time",
            "Std_ideal_time",
            "Avg_tps",
            "Std_tps",
            "Avg_rtps",
            "Std_rtps",
            "Avg_wtps",
            "Std_wtps",
            "Avg_ldavg_1",
            "Std_ldavg_1",
            "Avg_kbmemused",
            "Std_kbmemused",
            "Avg_num_Proc/s",
            "Std_num_proc/s",
            "Avg_num_cswch/s",
            "std_num_cswch/s",
            "OSSEC_alert",
            "OSSEC_alert_level",
            "Login_attempt",
            "Succesful_login",
            "File_activity",
            "Process_activity",
            "read_write_physical.process",
            "is_privileged"
            ]

bol_cols = ["anomaly_alert"]

features_to_keep = [
            "Duration",
            "Scr_bytes",
            "Des_bytes",
            "Scr_pkts",
            "Des_pkts",
            "Attack"
            ]

In [6]:
df_raw.head(5)

,Date,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3
0,9/01/2020,1578540956,192.168.2.199,49278,192.168.2.10,80,tcp,http,0.67369,13437,34924,1,0,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,105,18905,105,40392,TRUE,107658,210,311.7160712,159803.4704,0.5,0.5,0.300414275,0.699585725,9.207,5.55584206,10.994,1.356305275,4.864,1.873004004,0.311,0.224653066,74.624,8.245611196,12.297,10.38585004,8,10.50714043,4.297,2.723578712,2.146,0.102781321,915852.8,2507.97563,5.1,3.238826948,2806.2,158.7493622,1,5,0,0,0,0,0,0,Scanning_vulnerability,Reconnaissance,Attack
1,13/01/2020,1578871873,10.0.1.5,39769,131.236.3.92,53,udp,dns,0.000083,78,0,1,0,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,2,134,0,0,FALSE,212,2,24096.38554,2554216.867,1,0,1,0,12.326,6.674247823,2.895,1.604302029,3.778,2.099741889,2.741,5.327471164,78.263,8.869399134,6.6,5.713142743,0,0,6.6,5.713142743,0.612,0.037629775,910498.4,4044.112046,1,0.894427191,1561.6,247.0802299,0,0,0,0,0,0,0,0,Normal,Normal,Normal
2,9/01/2020,1578522486,172.24.1.80,59050,172.24.1.1,53,udp,dns,0.000132,38,38,1,0,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,1,66,1,66,FALSE,208,2,15151.51515,1575757.576,0.5,0.5,0.5,0.5,6.931,6.416007248,0.706,0.408905857,1.693,0.771635277,2.423,3.829809525,88.245,7.112108337,37.4,40.19004852,30.1,39.79811553,7.3,3.1,0.55,0.02,921020.4,2139.652645,1,0,1603.3,294.1390997,0,0,0,0,0,0,0,0,Normal,Normal,Normal
3,27/02/2020,1582757640,192.168.2.196,37966,192.168.2.10,1880,tcp,websocket,9.378481,1121,484,1,0,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,8,1545,6,804,FALSE,3954,14,1.492779055,421.6034558,0.571428571,0.428571429,0.67425392,0.32574608,10.244,7.932040343,1.965,1.514168088,3.941,1.709055002,0.509,0.425522032,83.339,10.20608686,29,26.57442379,23.3,26.12680616,5.7,6.229767251,0.708,0.026,915874.8,977.8851466,7.2,5.325410782,1685.3,467.8737116,0,0,1,1,1,1,1,1,Normal,Normal,Normal
4,16/12/2019,1576452612,172.24.1.80,38233,172.24.1.1,53,udp,dns,0.000074,-,-,1,0,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,-,-,-,-,FALSE,-,-,-,-,-,-,-,-,5.181,5.980458929,2.365,1.284462923,2.415,1.401151312,1.803,4.478256469,88.239,8.090017862,9.8,1.326649916,0.1,0.3,9.7,1.268857754,0.536,0.030066593,918213.2,2342.93836,0.6,0.489897949,1456.5,346.8847791,0,0,0,0,0,0,0,0,Normal,Normal,Normal


In [7]:
df_raw.describe()

,Date,Timestamp,Scr_IP,Scr_port,Des_IP,Des_port,Protocol,Service,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class1,class2,class3
count,820503,820537,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834,820834
unique,49,218989,36,57460,96,6444,4,17,330207,21054,51899,2,231,2,2,2,2,2,1,1,1211,32872,1556,55574,3,62464,2216,303353,339658,19995,19995,72716,72716,12635,51646,4551,50541,5174,50456,10157,30703,19705,51725,3736,21771,2254,16492,2017,11124,2814,3450,28144,51523,1061,3649,14986,51588,2,6,2,2,2,2,2,2,19,10,2
top,24/02/2020,1578542119,192.168.2.199,5353,192.168.2.10,80,tcp,http,-,-,-,1,0,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,1,-,1,0,FALSE,-,2,-,-,0.5,0.5,-,-,6.478,6.07937176,1.472,1.403424383,2.15,2.563215949,0.077,0.117647779,87.054,7.736539537,3,0,0,0,3,0,0.825,0.02,914762.8,2614.998616,0,0,1578.8,393.7089788,0,0,0,0,0,0,0,0,Normal,Normal,Normal
freq,147621,5631,156023,2889,312684,237640,422334,237738,75761,223871,223871,699041,820351,420448,497805,497546,711816,415483,820834,820834,249056,152743,208246,174182,740250,223872,260010,223871,223871,248977,248977,223872,223872,5776,5707,6071,5707,6365,5707,12923,13186,5754,5707,11726,11484,138135,138134,33190,32601,6766,37463,5755,5707,131979,147442,5804,5707,778819,778819,749171,752926,761200,753017,529184,753151,421417,421417,421417


In [8]:
#Cleaning
replacement_number = np.nan

#Drop time related features
df_cleaned = df_raw.drop(["Date","Timestamp"],axis = 1)


for col in num_cols:
    df_cleaned[col] = df_cleaned[col].apply(lambda x: convert_to_number(x,replacement_number))
for col in bol_cols:
    df_cleaned[col] = df_cleaned[col].replace({"-" : replacement_number})

print("IPV6 deleted adresses count:")
x = df_cleaned.loc[df_cleaned["Scr_IP"].str.contains(":",  na = False)].shape[0]
print(x)
print(x/df_raw.shape[0])
print("no data entries count:")
x = df_cleaned.loc[df_cleaned["Protocol"] == "?"].shape[0]
print(x)
print(x/df_raw.shape[0])

df_cleaned.loc[:,"Protocol"] = df_cleaned["Protocol"].apply(lambda x: x.upper())
df_cleaned = df_cleaned.drop(df_cleaned.loc[df_cleaned["Scr_IP"].str.contains(":",  na = False)].index) # IPV 6 adresses deleted
df_cleaned = df_cleaned.loc[df_cleaned["Protocol"] != "?"] #Attack entries that contain no information deleted



IPV6 deleted adresses count:
592
0.000721217688351116
no data entries count:
154
0.00018761406082106735


In [9]:
print("nan count:")
x = df_cleaned.shape[0]-df_cleaned.dropna().shape[0]
print(x)
print(x/df_raw.shape[0])

nan count:
224433
0.27342069163801697


In [10]:
#Replace boolean features
for col in bol_cols:
    df_cleaned[col] = df_cleaned[col].replace({np.nan : False})

In [11]:
#Reload dataframe for automatic type detection
stream = io.StringIO()
df_cleaned.to_csv(stream, index=False)
stream.seek(0)
df_reloaded = pd.read_csv(stream)
stream.close()

In [12]:
df_reloaded.columns

Index(['Scr_IP', 'Scr_port', 'Des_IP', 'Des_port', 'Protocol', 'Service',
       'Duration', 'Scr_bytes', 'Des_bytes', 'Conn_state', 'missed_bytes',
       'is_syn_only', 'Is_SYN_ACK', 'is_pure_ack', 'is_with_payload',
       'FIN or RST', 'Bad_checksum', 'is_SYN_with_RST', 'Scr_pkts',
       'Scr_ip_bytes', 'Des_pkts', 'Des_ip_bytes', 'anomaly_alert',
       'total_bytes', 'total_packet', 'paket_rate', 'byte_rate',
       'Scr_packts_ratio', 'Des_pkts_ratio', 'Scr_bytes_ratio',
       'Des_bytes_ratio', 'Avg_user_time', 'Std_user_time', 'Avg_nice_time',
       'Std_nice_time', 'Avg_system_time', 'Std_system_time',
       'Avg_iowait_time', 'Std_iowait_time', 'Avg_ideal_time',
       'Std_ideal_time', 'Avg_tps', 'Std_tps', 'Avg_rtps', 'Std_rtps',
       'Avg_wtps', 'Std_wtps', 'Avg_ldavg_1', 'Std_ldavg_1', 'Avg_kbmemused',
       'Std_kbmemused', 'Avg_num_Proc/s', 'Std_num_proc/s', 'Avg_num_cswch/s',
       'std_num_cswch/s', 'OSSEC_alert', 'OSSEC_alert_level', 'Login_attempt',
       

In [13]:
df_reloaded["Protocol"].unique()

array(['TCP', 'UDP', 'ICMP'], dtype=object)

In [14]:
#Preprocessing
df_processed = df_reloaded

#Dropping
df_processed = df_processed.drop(["class1"],axis = 1)
df_processed = df_processed.drop(["class2"],axis = 1)

#Imputing
df_processed = impute_by_target_col(df_processed,"Protocol",features_to_keep)

#Renaming for Netflow Datasets
df_processed = df_processed.rename(columns={
    "Protocol":"PROTOCOL",
    "Duration":"FLOW_DURATION_MILLISECONDS",
    "Scr_bytes":"OUT_BYTES",
    "Des_bytes":"IN_BYTES",
    "Scr_pkts":"OUT_PKTS",
    "Des_pkts":"IN_PKTS",
    "Scr_IP":"IPV4_SRC_ADDR",
    "Des_IP":"IPV4_DST_ADDR",
    "Scr_port":"L4_SRC_PORT",
    "Des_port":"L4_DST_PORT"
})

#Converting Attack Label to Bool
df_processed["class3"] = df_processed["class3"].apply(lambda x: 1 if x == "Attack" else 0)
df_processed = df_processed.rename(columns={"class3":"Label"})

In [15]:
df_processed.head(5)

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,Service,FLOW_DURATION_MILLISECONDS,OUT_BYTES,IN_BYTES,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,OUT_PKTS,Scr_ip_bytes,IN_PKTS,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,Label
0,192.168.2.199,49278.0,192.168.2.10,80.0,TCP,http,0.673690,13437.00000,34924.000000,1,0.0,True,True,True,True,True,False,False,105.000000,18905.0,105.000000,40392.0,True,107658.0,210.0,311.716071,159803.470400,0.500000,0.500000,0.300414,0.699586,9.207,5.555842,10.994,1.356305,4.864,1.873004,0.311,0.224653,74.624,8.245611,12.297,10.385850,8.0,10.507140,4.297,2.723579,2.146,0.102781,915852.8,2507.975630,5.1,3.238827,2806.2,158.749362,1,5,0,0,0,0,0,0,1
1,192.168.2.196,37966.0,192.168.2.10,1880.0,TCP,websocket,9.378481,1121.00000,484.000000,1,0.0,True,True,True,True,True,False,False,8.000000,1545.0,6.000000,804.0,False,3954.0,14.0,1.492779,421.603456,0.571429,0.428571,0.674254,0.325746,10.244,7.932040,1.965,1.514168,3.941,1.709055,0.509,0.425522,83.339,10.206087,29.000,26.574424,23.3,26.126806,5.700,6.229767,0.708,0.026000,915874.8,977.885147,7.2,5.325411,1685.3,467.873712,0,0,1,1,1,1,1,1,0
2,192.168.10.155,50502.0,192.168.2.10,80.0,TCP,http,5.153928,460.00000,671.000000,1,0.0,True,True,True,True,True,False,False,12.000000,1560.0,10.000000,1878.0,False,4569.0,22.0,4.268589,886.508310,0.545455,0.454545,0.442110,0.557890,10.579,6.866914,3.273,2.366141,2.638,1.040921,0.279,0.210829,83.233,9.579212,19.300,18.423083,8.8,10.205881,10.500,13.529597,0.524,0.028000,910370.8,4006.881700,0.9,0.538516,1366.9,304.062642,0,0,0,0,0,0,1,0,0
3,172.24.1.213,3467.0,161.69.36.37,80.0,TCP,http,3.011189,0.00000,0.000000,1,0.0,True,False,False,False,False,False,False,2.000000,104.0,0.000000,0.0,False,104.0,2.0,0.664189,34.537852,1.000000,0.000000,1.000000,0.000000,11.505,9.762796,1.795,0.970775,3.669,1.543240,2.928,7.630635,80.104,14.572305,22.400,20.323386,8.9,17.131550,13.500,12.531959,0.624,0.110109,912787.6,3127.432404,1.2,0.748331,1519.4,185.391046,0,0,0,0,0,0,0,0,0
4,192.168.2.199,42465.0,192.168.2.10,1880.0,TCP,websocket,0.003863,16554.93363,-25506.464239,1,0.0,True,True,True,True,True,False,False,-441.855635,NaN,146.869271,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.855,6.155793,12.885,3.109682,8.039,2.830076,2.735,5.399112,65.484,10.616435,18.800,16.521501,7.5,8.546929,11.300,12.814445,0.913,0.021000,914442.8,1957.066417,13.8,7.997500,2889.6,717.982200,0,0,0,0,0,0,0,0,1


In [16]:
df_processed.describe()

,L4_SRC_PORT,L4_DST_PORT,FLOW_DURATION_MILLISECONDS,OUT_BYTES,IN_BYTES,Conn_state,missed_bytes,OUT_PKTS,Scr_ip_bytes,IN_PKTS,Des_ip_bytes,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,Label
count,820088.000000,820088.000000,820088.000000,8.200880e+05,8.200880e+05,820088.000000,8.200880e+05,820088.000000,6.676080e+05,820088.000000,6.676090e+05,5.965790e+05,667609.000000,5.965800e+05,5.965790e+05,667609.000000,667609.000000,596579.000000,596579.000000,819589.000000,819609.000000,819609.000000,819609.000000,819604.000000,819604.000000,819604.000000,819604.000000,819609.000000,819609.000000,819596.000000,819596.000000,819596.000000,819596.000000,819596.000000,819596.000000,819562.000000,819562.000000,818976.000000,818976.000000,819609.000000,819609.000000,819609.000000,819609.000000,820088.000000,820088.000000,820088.000000,820088.000000,820088.000000,820088.000000,820088.000000,820088.000000,820088.000000
mean,45808.197022,2240.209685,9.698792,2.031048e+03,5.506129e+04,0.851720,3.564816e+01,26.031087,3.082449e+03,37.177036,5.331708e+04,1.215924e+05,58.346737,2.999369e+04,3.193058e+06,0.643616,0.356384,0.572309,0.427691,10.112034,5.855865,2.706363,1.547597,4.576969,1.634279,2.157225,3.577942,80.447911,9.481081,15.809487,13.034756,7.095183,9.547381,8.714321,5.905881,0.995593,0.058123,914156.365189,2490.516874,2.693628,2.958004,3945.771652,623.004722,0.051232,0.267960,0.087202,0.082623,0.072532,0.082511,0.355633,0.082347,0.486854
std,12237.626323,4846.902489,114.672244,8.598969e+03,3.248828e+05,0.355378,3.981357e+03,312.928431,2.136636e+04,303.239935,3.219795e+05,6.769641e+05,584.087353,9.488537e+04,1.132178e+07,0.224373,0.224373,0.295219,0.295219,8.964108,2.397998,2.002380,0.841908,4.264697,0.878964,3.650195,5.710527,13.569503,4.467291,15.560577,14.162770,14.950015,14.892822,4.258546,5.128661,0.646491,0.081763,12382.925578,2366.669624,580.524370,820.973276,5268.494992,863.336952,0.220471,1.178535,0.282130,0.275311,0.259368,0.275141,0.478705,0.274893,0.499827
min,3.000000,0.000000,-709.376650,-4.605074e+04,-1.872586e+06,0.000000,0.000000e+00,-1753.826502,0.000000e+00,-1837.099938,0.000000e+00,5.600000e+01,1.000000,2.337215e-02,1.562475e+00,0.000000,0.000000,0.000000,0.000000,0.228000,0.125064,0.000000,0.000000,0.507000,-0.198966,0.000000,-0.498306,0.000000,0.000000,-45.500000,-19.152128,0.000000,0.000000,-46.300000,-42.037441,0.020000,-0.163691,528121.200000,5.600000,-11.900000,-24.694497,-861.400000,-1918.070330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,39134.000000,53.000000,0.000186,3.700000e+01,0.000000e+00,1.000000,0.000000e+00,1.000000,7.500000e+01,0.000000,0.000000e+00,2.040000e+02,2.000000,5.384286e+00,2.927566e+03,0.500000,0.000000,0.414557,0.129909,4.308000,4.354195,1.695000,1.045053,2.155000,1.072269,0.207000,0.168570,76.228000,6.754729,9.100000,5.325411,0.100000,0.300000,4.697000,1.417745,0.533000,0.022271,913080.800000,1617.490748,0.499000,0.400000,1333.300000,233.883817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,48346.000000,80.000000,0.024902,2.320000e+02,7.200000e+01,1.000000,0.000000e+00,2.000000,5.900000e+02,1.000000,7.300000e+01,1.470000e+03,3.000000,3.426513e+02,1.571725e+05,0.529412,0.470588,0.507043,0.492957,6.650000,5.691580,2.166000,1.387738,2.753000,1.426613,0.457000,0.409141,85.708000,8.666752,13.800000,10.336827,3.800000,6.103278,9.200000,5.079370,0.795000,0.035000,915435.600000,2134.767022,0.800000,0.629314,1526.700000

In [17]:
#Export
df_processed.to_csv(df_name+"_pre_aligned.csv",index=False)